<a href="https://colab.research.google.com/github/csantelices/DataScience/blob/master/practico_pig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apache Pig

## Instalación y copia de datos

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving hadoop_colab_installer.py to hadoop_colab_installer.py
User uploaded file "hadoop_colab_installer.py" with length 13498 bytes


In [2]:
exec(open('hadoop_colab_installer.py').read())

Active services:
2387 Jps
2006 DataNode
2188 NodeManager
2349 JobHistoryServer
1853 ResourceManager
1935 NameNode



In [3]:
!tail pig/tutorial/pigtmp/excite-small.log

1432F28A749F7BE6	970916220548	nesticle 0.40 download
1432F28A749F7BE6	970916221400	nes rom
1432F28A749F7BE6	970916221804	double dragon 3 rom
1432F28A749F7BE6	970916221859	emuland
98F5BBD3754D292F	970916082614	powwow.com
98F5BBD3754D292F	970916082616	powwow.com
98F5BBD3754D292F	970916082713	powwow.com tribal 
C5D01E05FF9CA265	970916155706	mary lou allgood
C5D01E05FF9CA265	970916155729	mary lou allgood
DB38E7AF26F3AD9A	970916114356	microsoft excel


In [4]:
!hdfs dfs -put pig/tutorial/pigtmp/excite-small.log /user/root/

## Funciones definidas por el usuario (UDFs)

In [5]:
%%writefile my_pig_udfs.py

from pig_util import outputSchema
import re
import array

@outputSchema('detected:boolean')
def empty_or_url_detector( query ):
  if query is None:
    return True
  query = query.tostring() # 'array' to 'str'
  if len( query ) == 0 or re.search( '(http:)|(https:)|(www.)', query ):
    return True
  return False

@outputSchema('hour:chararray')
def extract_hour( time ):
  time = time.tostring() # 'array' to 'str'
  return time[6:8]

Writing my_pig_udfs.py


## Pig Latin

In [7]:
!pig

21/08/04 05:44:43 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
21/08/04 05:44:43 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
21/08/04 05:44:43 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2021-08-04 05:44:43,273 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2021-08-04 05:44:43,274 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1628055883271.log
2021-08-04 05:44:43,309 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2021-08-04 05:44:43,805 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2021-08-04 05:44:43,805 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: hdfs://localhost:9000
2021-08-04 05:44:44,695 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the session:

In [8]:
!hdfs dfs -ls

Found 2 items
-rw-r--r--   1 root supergroup     208348 2021-08-04 05:16 excite-small.log
drwxr-xr-x   - root supergroup          0 2021-08-04 06:05 num_queries


In [9]:
!hdfs dfs -ls num_queries

Found 2 items
-rw-r--r--   1 root supergroup          0 2021-08-04 06:05 num_queries/_SUCCESS
-rw-r--r--   1 root supergroup     175150 2021-08-04 06:05 num_queries/part-r-00000


In [10]:
!hdfs dfs -cat num_queries/part-r-00000

A8A0674EA33D1249,00,"iron and steel" rusia ukraine,00,99
8223F74BED5A061A,00,diablo cheats ,00,99
8223F74BED5A061A,00,diablo,00,99
A8A0674EA33D1249,00,"iron and steel" rusia ukraine,00,99
E9CE7EE37511E710,00,jullie bell,00,99
72270DEAFE0BF9FC,00,fin6060a,00,99
A8A0674EA33D1249,00,"iron and steel" rusia ukraine,00,99
A8A0674EA33D1249,00,"iron and steel" rusia ukraine,00,99
8E1A8EA81FEA8A30,00,skolestil,00,99
A8A0674EA33D1249,00,"iron and steel" rusia ukraine,00,99
D2E8CBAEF95A890B,00,spice adult,00,99
A8A0674EA33D1249,00,"iron and steel" rusia ukraine,00,99
33ADF7360366F307,00,warez and adobe and illustrator,00,99
D2E8CBAEF95A890B,00,spice,00,99
D2E8CBAEF95A890B,00,spice,00,99
D2E8CBAEF95A890B,00,spice,00,99
AE5E2B48ED103FA0,00,free pictures,00,99
D2E8CBAEF95A890B,00,?where=excite&what=,00,99
D2E8CBAEF95A890B,00,?where=excite&what=,00,99
E9CE7EE37511E710,00,jullie bell,00,99
E9CE7EE37511E710,00,jullie bell,00,99
BED75271605EBD0C,00,yahoo chat,00,99
E9CE7EE37511E710,00,jullie bell,00,99


In [11]:
%%writefile practico_pig.pig

REGISTER 'my_pig_udfs.py' USING jython AS my_pig_udfs;

raw = LOAD 'excite-small.log' USING PigStorage('\t') AS (user_id, time, query);

clean = FILTER raw BY my_pig_udfs.empty_or_url_detector(query) == FALSE;

houred = FOREACH clean GENERATE user_id, my_pig_udfs.extract_hour(time) as (hour:chararray), query;

grouped_hour = GROUP houred BY hour;

num_queries = FOREACH grouped_hour GENERATE $0 AS hour, COUNT($1) AS count;

DUMP num_queries;

final_result = JOIN houred BY hour, num_queries BY hour;

STORE final_result INTO 'num_queries' USING PigStorage( ',' );

Writing practico_pig.pig


In [12]:
!pig -f practico_pig.pig

21/08/04 06:11:20 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
21/08/04 06:11:20 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
21/08/04 06:11:20 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2021-08-04 06:11:21,039 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2021-08-04 06:11:21,039 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1628057481037.log
2021-08-04 06:11:21,683 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2021-08-04 06:11:21,768 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2021-08-04 06:11:21,768 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: hdfs://localhost:9000
2021-08-04 06:11:22,540 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the session: